This notebook will be used to populate ZV2017 data frame with values from the nearest polygons from other years (2018, 2019).

Based on the nearest indices and nearest distance (from nearest_ids shape files), we can split the Data Frame into three parts:

1. Crop types with less than 1000 datapoints - all fields are NaN values

2. Polygons with nearest distance of less than 10 m - copy the value of the nearest data point

3. Polygons with nearest distance of more than 10 m - calculate mean value of three nearest data points

# 1. Monthly values of S2-indices

## 1.1 Year 2018

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

Load the `nearest_ids` data:

In [2]:
%%time
# Nearest IDs shapefile
df0 = pd.read_csv(".\\results_ids\\ZV2017_nearest_ids_18_ajda.csv")

print(df0.shape)
df0.head(2)

(664138, 8)
Wall time: 723 ms


,id_17,GERK_PID,POLJINA_ID,SIFRA_KMRS,dist0,near0,near1,near2
0,0,2537988,2104906,801,72.154793,133377.0,133640.0,468680.0
1,1,2537988,2104908,5,31.536706,133193.0,455175.0,439259.0


Load 2018 data:

In [3]:
%%time
# Monthly S2-indices for ear 2018
df1 = pd.read_csv(".\\results_monthly\\ZV2018_s2-indices_monthly.csv")

print(df1.shape)
df1.head(2)

(680698, 29)
Wall time: 5.47 s


,id_18,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,ndvi_05,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0,0.287490,0.573519,0.471059,0.546702,0.568249,0.500756,0.411956,0.480016,0.748722,...,-0.614697,-0.585942,-0.565263,0.304475,0.559818,0.479496,0.542311,0.566786,0.513388,0.437795
1,1,0.329425,0.386938,0.633417,0.630591,0.597212,0.383917,0.366518,0.545778,0.563321,...,-0.614866,-0.396175,-0.531713,0.351091,0.401721,0.600088,0.609367,0.582778,0.407802,0.397842


Prepare output table (same format as 2017 monthly):

In [4]:
%%time

# Open monthly S2-indices for year 2017 and set all values to nan
df_out = pd.read_csv(".\\results_monthly\\ZV2017_s2-indices_monthly.csv")

# Set all values to NaN
df_out.iloc[:, 1:] = np.nan

print(df_out.shape)
df_out.head(2)

(664138, 29)
Wall time: 5.36 s


,id_17,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,ndvi_05,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Split the dataset into "FAR" and "NEAR" sub-datasets. Those that are further than 10m (one pixel) away will be calculated as mean of the nearest 3 samples. Those that are near will be assume the value of the nearest field.

In [5]:
# Create filters for the two groups
filter_far = df0["dist0"] >= 10
filter_near = df0["dist0"] < 10

In [6]:
# Find all FAR and "melt" them
melted = df0.loc[filter_far, ["id_17", "dist0", "near0", "near1", "near2"]].melt(id_vars=["id_17", "dist0"])
melted

,id_17,dist0,variable,value
0,0,72.154793,near0,133377.0
1,1,31.536706,near0,133193.0
2,2,608.886482,near0,292837.0
3,3,103.529690,near0,133367.0
4,9,820.491920,near0,209673.0
...,...,...,...,...
582466,664112,64.439271,near2,566708.0
582467,664116,168.048932,near2,569205.0
582468,664118,80.278087,near2,449630.0
582469,664119,31.410108,near2,167270.0


In [7]:
# List of all column labels from 2018 data frame (without id_18)
my_columns = df1.iloc[:, 1:].columns.to_list()

# Create new data frame based on the index from melted
extracted_values = df1.loc[melted.value.to_numpy(), my_columns].reset_index(drop=True)
extracted_values

,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,ndvi_05,ndvi_06,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0.233672,0.216744,0.310050,0.450514,0.326207,0.147023,0.095923,0.411656,0.379127,0.485982,...,-0.478299,-0.334952,-0.043652,0.256460,0.238042,0.329590,0.466908,0.346441,0.167066,0.108685
1,0.177843,0.194499,0.347213,0.586015,0.461673,0.170602,0.035172,0.302347,0.359710,0.523425,...,-0.534351,-0.321338,0.003836,0.197030,0.214629,0.363716,0.581012,0.473608,0.193117,0.040262
2,0.373359,0.578273,0.545290,0.536742,0.533152,0.389640,0.395918,0.546728,0.780214,0.744096,...,-0.584612,-0.437081,-0.471409,0.387592,0.570883,0.542532,0.540249,0.535898,0.404946,0.415841
3,0.325917,0.366484,0.303504,0.426817,0.393938,0.237727,0.286898,0.513839,0.582476,0.523208,...,-0.507760,-0.385635,-0.339645,0.345702,0.383532,0.327394,0.445044,0.412930,0.261249,0.308934
4,0.321683,0.354035,0.407175,0.450385,0.445054,0.450840,0.504993,0.548666,0.498844,0.589589,...,-0.479866,-0.461179,-0.520971,0.344969,0.368364,0.416404,0.456163,0.453061,0.451039,0.510185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582466,0.249411,0.365032,0.455745,0.644431,0.567964,0.351274,0.371675,0.415244,0.563348,0.641359,...,-0.671935,-0.432536,-0.452265,0.271198,0.381740,0.462217,0.626456,0.565904,0.372891,0.396568
582467,0.551925,0.344896,0.261731,0.422957,0.493079,0.321154,0.378404,0.699922,0.542909,0.410902,...,-0.537141,-0.410989,-0.421224,0.542252,0.366409,0.280690,0.433671,0.497712,0.342141,0.396297
582468,0.327126,0.365229,0.418366,0.514260,0.402364,0.454754,0.433428,0.490835,0.543859,0.580724,...,-0.516638,-0.547844,-0.523445,0.343595,0.382073,0.428434,0.515110,0.421268,0.470912,0.450251
582469,0.169500,0.258320,0.364577,0.599657,0.545008,0.218865,0.188468,0.285744,0.381088,0.587937,...,-0.627970,-0.375870,-0.334873,0.188075,0.276929,0.381941,0.588829,0.545406,0.241105,0.203885


In [8]:
# Join the new data frame with the melted data frame
before_pivot = pd.concat([melted, extracted_values], axis=1)
before_pivot = before_pivot.drop(columns=["value", "dist0"])
before_pivot.head(3)

,id_17,variable,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0,near0,0.233672,0.216744,0.310050,0.450514,0.326207,0.147023,0.095923,0.411656,...,-0.478299,-0.334952,-0.043652,0.256460,0.238042,0.329590,0.466908,0.346441,0.167066,0.108685
1,1,near0,0.177843,0.194499,0.347213,0.586015,0.461673,0.170602,0.035172,0.302347,...,-0.534351,-0.321338,0.003836,0.197030,0.214629,0.363716,0.581012,0.473608,0.193117,0.040262
2,2,near0,0.373359,0.578273,0.545290,0.536742,0.533152,0.389640,0.395918,0.546728,...,-0.584612,-0.437081,-0.471409,0.387592,0.570883,0.542532,0.540249,0.535898,0.404946,0.415841


In [9]:
# Pivot the table back to the original shape by calculating mean of the three nearest data points and insert into output df
df_out.loc[filter_far, my_columns] = before_pivot.pivot_table(
    values=my_columns,
    index=['id_17'],
    aggfunc=np.nanmean
)

df_out

,id_17,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,ndvi_05,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0,0.310288,0.336024,0.388160,0.343497,0.298037,0.274945,0.142445,0.496785,0.501972,...,-0.426997,-0.361220,-0.118119,0.328986,0.347970,0.399603,0.365345,0.318758,0.286160,0.162205
1,1,0.184245,0.206430,0.381287,0.582966,0.480260,0.192997,0.045441,0.319610,0.343977,...,-0.561179,-0.322342,0.008337,0.204212,0.224641,0.393981,0.578173,0.490874,0.217317,0.051727
2,2,0.442552,0.417792,0.435727,0.512223,0.419965,0.407048,0.354278,0.609960,0.591340,...,-0.503948,-0.465827,-0.394135,0.448030,0.424985,0.442461,0.514929,0.433656,0.420565,0.374315
3,3,0.366594,0.391448,0.284961,0.353611,0.387828,0.218679,0.242736,0.577629,0.615001,...,-0.500857,-0.352424,-0.299285,0.384893,0.406886,0.307465,0.373147,0.406821,0.241211,0.262006
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664133,664133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664134,664134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664135,664135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664136,664136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Find all NEAR and "clean" the data frame
near = df0.loc[filter_near, ["id_17", "dist0", "near0"]]
near

,id_17,dist0,near0
4,4,1.590166e-05,126401.0
5,5,1.729459e-08,126403.0
6,6,2.876625e-08,126404.0
7,7,2.805786e-08,126405.0
8,8,2.587472e+00,126407.0
...,...,...,...
664133,664133,5.507934e-09,121004.0
664134,664134,5.070753e-09,121005.0
664135,664135,6.691366e-09,121006.0
664136,664136,4.831258e-09,121007.0


In [11]:
extracted_near = df1.loc[near.near0.to_numpy(), my_columns].reset_index(drop=True)
extracted_near = extracted_near.set_index(df0.loc[filter_near, "id_17"].to_numpy())

In [12]:
df_out

,id_17,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,ndvi_05,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0,0.310288,0.336024,0.388160,0.343497,0.298037,0.274945,0.142445,0.496785,0.501972,...,-0.426997,-0.361220,-0.118119,0.328986,0.347970,0.399603,0.365345,0.318758,0.286160,0.162205
1,1,0.184245,0.206430,0.381287,0.582966,0.480260,0.192997,0.045441,0.319610,0.343977,...,-0.561179,-0.322342,0.008337,0.204212,0.224641,0.393981,0.578173,0.490874,0.217317,0.051727
2,2,0.442552,0.417792,0.435727,0.512223,0.419965,0.407048,0.354278,0.609960,0.591340,...,-0.503948,-0.465827,-0.394135,0.448030,0.424985,0.442461,0.514929,0.433656,0.420565,0.374315
3,3,0.366594,0.391448,0.284961,0.353611,0.387828,0.218679,0.242736,0.577629,0.615001,...,-0.500857,-0.352424,-0.299285,0.384893,0.406886,0.307465,0.373147,0.406821,0.241211,0.262006
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664133,664133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664134,664134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664135,664135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664136,664136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_out.loc[filter_near, my_columns] = extracted_near
df_out

,id_17,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,ndvi_05,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0,0.310288,0.336024,0.388160,0.343497,0.298037,0.274945,0.142445,0.496785,0.501972,...,-0.426997,-0.361220,-0.118119,0.328986,0.347970,0.399603,0.365345,0.318758,0.286160,0.162205
1,1,0.184245,0.206430,0.381287,0.582966,0.480260,0.192997,0.045441,0.319610,0.343977,...,-0.561179,-0.322342,0.008337,0.204212,0.224641,0.393981,0.578173,0.490874,0.217317,0.051727
2,2,0.442552,0.417792,0.435727,0.512223,0.419965,0.407048,0.354278,0.609960,0.591340,...,-0.503948,-0.465827,-0.394135,0.448030,0.424985,0.442461,0.514929,0.433656,0.420565,0.374315
3,3,0.366594,0.391448,0.284961,0.353611,0.387828,0.218679,0.242736,0.577629,0.615001,...,-0.500857,-0.352424,-0.299285,0.384893,0.406886,0.307465,0.373147,0.406821,0.241211,0.262006
4,4,0.431386,0.616977,0.504878,0.445114,0.385201,0.499945,0.392365,0.600698,0.784408,...,-0.526163,-0.585877,-0.462735,0.438282,0.599068,0.507820,0.449248,0.406548,0.510420,0.400363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664133,664133,0.478952,0.619168,0.741609,0.508880,0.527436,0.686529,0.686529,0.611792,0.729346,...,-0.523196,-0.636928,-0.636928,0.468234,0.591041,0.689845,0.497784,0.515587,0.658304,0.658304
664134,664134,0.740605,0.556043,0.769581,0.604529,0.638880,0.482598,0.482598,0.864967,0.645474,...,-0.600412,-0.410304,-0.410304,0.693565,0.530756,0.708379,0.580835,0.610115,0.460249,0.460249
664135,664135,0.524517,0.737883,0.551391,0.690127,0.483002,0.524915,0.524915,0.695863,0.838841,...,-0.499203,-0.575823,-0.575823,0.519488,0.686580,0.543963,0.652788,0.483224,0.506696,0.506696
664136,664136,0.525806,0.481272,0.723050,0.344661,0.543307,0.512938,0.512938,0.714084,0.651296,...,-0.605693,-0.477379,-0.477379,0.523972,0.480405,0.677653,0.353587,0.543654,0.505275,0.505275


In [14]:
# Save to csv
df_out.to_csv(f".\\results_monthly_nearest\\ZV2017-2018_s2-indices_monthly_ajda.csv", index=False)

## 1.2 Year 2019

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
%%time
# Nearest IDs shapefile
df0 = pd.read_csv(".\\results_ids\\ZV2017_nearest_ids_19_ajda.csv")

print(df0.shape)
df0.head(2)

(664138, 8)
Wall time: 729 ms


,id_17,GERK_PID,POLJINA_ID,SIFRA_KMRS,dist0,near0,near1,near2
0,0,2537988,2104906,801,44.375468,8378.0,282621.0,191282.0
1,1,2537988,2104908,5,23.064102,7685.0,427672.0,7732.0


In [19]:
%%time
# Monthly S2-indices for year 2019
df1 = pd.read_csv(".\\results_monthly\\KMRS2019_s2-indices_monthly.csv")

print(df1.shape)
df1.head(2)

(626062, 29)
Wall time: 3.79 s


,id_19,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,ndvi_05,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0,0.446085,0.414179,0.555391,0.442216,0.513699,0.622103,0.416238,0.681025,0.770184,...,-0.537258,-0.618994,-0.419042,0.458718,0.441694,0.551193,0.445762,0.511181,0.611007,0.431188
1,1,0.549344,0.206892,0.567524,0.511613,0.582250,0.546557,0.411344,0.766354,0.334661,...,-0.554503,-0.487522,-0.434779,0.547855,0.225171,0.560088,0.500999,0.571696,0.529209,0.427107


In [20]:
%%time

# Open monthly S2-indices for year 2017 and set all values to nan
df_out = pd.read_csv(".\\results_monthly\\ZV2017_s2-indices_monthly.csv")

# Set all values to NaN
df_out.iloc[:, 1:] = np.nan

print(df_out.shape)
df_out.head(2)

(664138, 29)
Wall time: 4.04 s


,id_17,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,ndvi_05,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Create filters for the two groups
filter_far = df0["dist0"] >= 10
filter_near = df0["dist0"] < 10

In [22]:
# Find all FAR and "melt" them
melted = df0.loc[filter_far, ["id_17", "dist0", "near0", "near1", "near2"]].melt(id_vars=["id_17", "dist0"])
melted

,id_17,dist0,variable,value
0,0,44.375468,near0,8378.0
1,1,23.064102,near0,7685.0
2,2,44.696477,near0,4577.0
3,3,35.919025,near0,7693.0
4,9,114.809007,near0,544036.0
...,...,...,...,...
605566,664112,111.920322,near2,623122.0
605567,664115,33.804681,near2,564788.0
605568,664116,119.505800,near2,615556.0
605569,664119,132.200676,near2,69315.0


In [23]:
# List of all column labels from 2018 data frame (without id_18)
my_columns = df1.iloc[:, 1:].columns.to_list()

# Create new data frame based on the index from melted
extracted_values = df1.loc[melted.value.to_numpy(), my_columns].reset_index(drop=True)
extracted_values

,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,ndvi_05,ndvi_06,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0.469326,0.453179,0.438142,0.612780,0.540398,0.542068,0.278189,0.729849,0.766181,0.634637,...,-0.523232,-0.540311,-0.313419,0.483497,0.467227,0.447128,0.595515,0.533091,0.542600,0.304582
1,0.437025,0.472096,0.413314,0.534774,0.499175,0.450998,0.153985,0.713830,0.776745,0.648195,...,-0.388573,-0.503716,-0.318497,0.455931,0.488473,0.430508,0.532649,0.499500,0.462223,0.173586
2,0.452984,0.484241,0.368393,0.267160,0.286023,0.440358,0.173512,0.744746,0.812475,0.597354,...,-0.372441,-0.503509,-0.338208,0.471201,0.498638,0.388058,0.287882,0.307052,0.455142,0.196582
3,0.483322,0.587703,0.367070,0.281950,0.355539,0.378423,0.199469,0.765808,0.835780,0.607653,...,-0.431331,-0.478093,-0.300080,0.497436,0.583552,0.389035,0.305954,0.374255,0.394808,0.218742
4,0.398582,0.441304,0.405463,0.291259,0.413523,0.377553,0.492276,0.570583,0.631253,0.575197,...,-0.439944,-0.447913,-0.546129,0.413365,0.453915,0.417597,0.307453,0.424601,0.392741,0.496149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605566,0.383320,0.426321,0.445451,0.508452,0.481053,0.538977,0.371706,0.603969,0.639839,0.654650,...,-0.569199,-0.599646,-0.459154,0.402886,0.440825,0.455338,0.508084,0.492712,0.545242,0.391996
605567,0.519603,0.472808,0.585814,0.533954,0.549101,0.524673,0.428436,0.753595,0.724067,0.782431,...,-0.614487,-0.598136,-0.483677,0.524997,0.481532,0.577208,0.536657,0.548738,0.529535,0.440809
605568,0.411617,0.435070,0.309717,0.387936,0.358470,0.475472,0.455868,0.621846,0.660993,0.516921,...,-0.465014,-0.556522,-0.467776,0.427366,0.450177,0.331107,0.403840,0.376158,0.485552,0.456996
605569,0.630336,0.457722,0.317807,0.342000,0.386318,0.500182,0.359122,0.833069,0.596130,0.545939,...,-0.385317,-0.553081,-0.407018,0.615071,0.456427,0.338093,0.358035,0.400266,0.508837,0.372849


In [24]:
# Join the new data frame with the melted data frame
before_pivot = pd.concat([melted, extracted_values], axis=1)
before_pivot = before_pivot.drop(columns=["value", "dist0"])
before_pivot.head(3)

,id_17,variable,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0,near0,0.469326,0.453179,0.438142,0.612780,0.540398,0.542068,0.278189,0.729849,...,-0.523232,-0.540311,-0.313419,0.483497,0.467227,0.447128,0.595515,0.533091,0.542600,0.304582
1,1,near0,0.437025,0.472096,0.413314,0.534774,0.499175,0.450998,0.153985,0.713830,...,-0.388573,-0.503716,-0.318497,0.455931,0.488473,0.430508,0.532649,0.499500,0.462223,0.173586
2,2,near0,0.452984,0.484241,0.368393,0.267160,0.286023,0.440358,0.173512,0.744746,...,-0.372441,-0.503509,-0.338208,0.471201,0.498638,0.388058,0.287882,0.307052,0.455142,0.196582


In [25]:
# Pivot the table back to the original shape by calculating mean of the three nearest data points and insert into output df
df_out.loc[filter_far, my_columns] = before_pivot.pivot_table(
    values=my_columns,
    index=['id_17'],
    aggfunc=np.nanmean
)

df_out

,id_17,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,ndvi_05,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0,0.475333,0.516805,0.387616,0.424247,0.421510,0.388793,0.266754,0.739675,0.790046,...,-0.438106,-0.422847,-0.349613,0.488035,0.520556,0.403153,0.431758,0.429516,0.403453,0.292749
1,1,0.395264,0.275513,0.363924,0.594162,0.543531,0.474665,0.159731,0.571526,0.491134,...,-0.541907,-0.508676,-0.303776,0.399050,0.295574,0.379030,0.582267,0.537170,0.484087,0.178544
2,2,0.496445,0.542524,0.427945,0.395171,0.426065,0.413482,0.429798,0.728292,0.778659,...,-0.485900,-0.475875,-0.487826,0.503183,0.542357,0.440512,0.410291,0.435391,0.429631,0.433333
3,3,0.526974,0.604756,0.389379,0.274047,0.289784,0.385243,0.194265,0.794320,0.843655,...,-0.351762,-0.443663,-0.295520,0.533449,0.596431,0.406899,0.295780,0.308346,0.402116,0.214720
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664133,664133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664134,664134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664135,664135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664136,664136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Find all NEAR and "clean" the data frame
near = df0.loc[filter_near, ["id_17", "dist0", "near0"]]
near

,id_17,dist0,near0
4,4,0.358454,573041.0
5,5,0.354016,573045.0
6,6,0.380426,573046.0
7,7,0.380303,573423.0
8,8,2.397666,573043.0
...,...,...,...
664133,664133,0.072652,619492.0
664134,664134,0.070845,619493.0
664135,664135,0.073263,619494.0
664136,664136,0.253449,619495.0


In [27]:
extracted_near = df1.loc[near.near0.to_numpy(), my_columns].reset_index(drop=True)
extracted_near = extracted_near.set_index(df0.loc[filter_near, "id_17"].to_numpy())

In [28]:
df_out

,id_17,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,ndvi_05,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0,0.475333,0.516805,0.387616,0.424247,0.421510,0.388793,0.266754,0.739675,0.790046,...,-0.438106,-0.422847,-0.349613,0.488035,0.520556,0.403153,0.431758,0.429516,0.403453,0.292749
1,1,0.395264,0.275513,0.363924,0.594162,0.543531,0.474665,0.159731,0.571526,0.491134,...,-0.541907,-0.508676,-0.303776,0.399050,0.295574,0.379030,0.582267,0.537170,0.484087,0.178544
2,2,0.496445,0.542524,0.427945,0.395171,0.426065,0.413482,0.429798,0.728292,0.778659,...,-0.485900,-0.475875,-0.487826,0.503183,0.542357,0.440512,0.410291,0.435391,0.429631,0.433333
3,3,0.526974,0.604756,0.389379,0.274047,0.289784,0.385243,0.194265,0.794320,0.843655,...,-0.351762,-0.443663,-0.295520,0.533449,0.596431,0.406899,0.295780,0.308346,0.402116,0.214720
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664133,664133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664134,664134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664135,664135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664136,664136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_out.loc[filter_near, my_columns] = extracted_near
df_out

,id_17,evi2_04,evi2_05,evi2_06,evi2_07,evi2_08,evi2_09,evi2_10,ndvi_04,ndvi_05,...,ndwi_08,ndwi_09,ndwi_10,savi_04,savi_05,savi_06,savi_07,savi_08,savi_09,savi_10
0,0,0.475333,0.516805,0.387616,0.424247,0.421510,0.388793,0.266754,0.739675,0.790046,...,-0.438106,-0.422847,-0.349613,0.488035,0.520556,0.403153,0.431758,0.429516,0.403453,0.292749
1,1,0.395264,0.275513,0.363924,0.594162,0.543531,0.474665,0.159731,0.571526,0.491134,...,-0.541907,-0.508676,-0.303776,0.399050,0.295574,0.379030,0.582267,0.537170,0.484087,0.178544
2,2,0.496445,0.542524,0.427945,0.395171,0.426065,0.413482,0.429798,0.728292,0.778659,...,-0.485900,-0.475875,-0.487826,0.503183,0.542357,0.440512,0.410291,0.435391,0.429631,0.433333
3,3,0.526974,0.604756,0.389379,0.274047,0.289784,0.385243,0.194265,0.794320,0.843655,...,-0.351762,-0.443663,-0.295520,0.533449,0.596431,0.406899,0.295780,0.308346,0.402116,0.214720
4,4,0.464375,0.485903,0.290563,0.341317,0.386973,0.356566,0.316453,0.655518,0.653835,...,-0.337800,-0.349697,-0.341531,0.473376,0.488886,0.305686,0.358752,0.404077,0.381204,0.338421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664133,664133,0.716563,0.811111,0.545029,0.532057,0.477386,0.534711,0.382760,0.878355,0.878903,...,-0.478962,-0.518060,-0.435791,0.678615,0.740450,0.533343,0.530710,0.470313,0.529857,0.399561
664134,664134,0.720971,0.828751,0.496057,0.467434,0.495120,0.470899,0.430521,0.871465,0.895177,...,-0.469159,-0.426033,-0.448561,0.681963,0.753567,0.490692,0.472420,0.484949,0.465842,0.437361
664135,664135,0.521262,0.632875,0.426946,0.525122,0.478167,0.494546,0.392233,0.753579,0.818823,...,-0.481010,-0.483617,-0.438679,0.526449,0.615181,0.430345,0.524369,0.479338,0.499894,0.399929
664136,664136,0.604316,0.733242,0.559754,0.500993,0.488272,0.380353,0.563102,0.820573,0.877371,...,-0.521344,-0.417492,-0.616514,0.595122,0.689923,0.545984,0.506453,0.488057,0.397669,0.566238


In [30]:
# Save to csv
df_out.to_csv(f".\\results_monthly_nearest\\ZV2017-2019_s2-indices_monthly_ajda.csv", index=False)

# 2. Monthly values of S2-rgbn

## 2.1 Year 2018

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

Load the `nearest_ids` data:

In [ ]:
%%time
# Nearest IDs shapefile
df0 = pd.read_csv(".\\results_ids\\ZV2017_nearest_ids_18_ajda.csv")

print(df0.shape)
df0.head()

Split the nearest_IDs dataset into "FAR" and "NEAR" sub-datasets. Those that are further than 10m (one pixel) away will be calculated as mean of the nearest 3 samples. Those that are near will be assume the value of the nearest field.

In [ ]:
# Create filters for the full data frame
crop_types = [4, 5, 20, 30, 100, 208, 405, 699, 800]

f_far_out = (df0["SIFRA_KMRS"].isin(crop_types)) & (df0["dist0"] >= 10)
f_near_out = (df0["SIFRA_KMRS"].isin(crop_types)) & (df0["dist0"] < 10)

f_far_out

In [ ]:
# Filter the nearest_IDs data frame
df0 = df0[df0["SIFRA_KMRS"].isin(crop_types)]

# Create filters for the already filtered data frame
f_far_df0 = df0["dist0"] >= 10
f_near_df0 = df0["dist0"] < 10

df0.head()

Load 2018 data:

In [ ]:
%%time
# Monthly S2-indices for ear 2018
df1 = pd.read_csv(".\\results_monthly\\ZV2018_s2-rgbn_monthly.csv")
df1 = df1.set_index("id_18")

print(df1.shape)
df1.head(2)

Prepare output table (same format as 2017 monthly):

In [ ]:
%%time

# Open monthly S2-indices for year 2017 and set all values to nan
df_out = pd.read_csv(".\\results_monthly\\ZV2017_s2-rgbn_monthly.csv")

# Set all values to NaN
df_out.iloc[:, 1:] = np.nan

print(df_out.shape)
df_out.head(2)

### Calculate "far" values

In [ ]:
# Find all FAR and "melt" them
melted = df0.loc[f_far_out, ["id_17", "dist0", "near0", "near1", "near2"]].melt(id_vars=["id_17", "dist0"])
melted

In [ ]:
# List of all column labels from 2018 data frame (without id_18)
my_columns = df1.iloc[:, 0:].columns.to_list()
my_columns

In [ ]:
# Create new data frame based on the index from melted
extracted_values = df1.loc[melted.value.to_numpy(), my_columns].reset_index(drop=True)
extracted_values

In [ ]:
# Join the new data frame with the melted data frame
before_pivot = pd.concat([melted, extracted_values], axis=1)
before_pivot = before_pivot.drop(columns=["value", "dist0"])
before_pivot

In [ ]:
before_pivot.pivot_table(
    values=my_columns,
    index=['id_17'],
    aggfunc=np.nanmean
)

In [ ]:
# Pivot the table back to the original shape by calculating mean of the three nearest data points and insert into output df
df_out.loc[f_far_out, my_columns] = before_pivot.pivot_table(
    values=my_columns,
    index=['id_17'],
    aggfunc=np.nanmean
)

df_out.loc[f_far_out, my_columns]

### Calculate "near" values

In [ ]:
# Find all NEAR and "clean" the data frame
near = df0.loc[f_near_out, ["id_17", "dist0", "near0"]]
near

In [ ]:
extracted_near = df1.loc[near.near0.to_numpy(), my_columns].reset_index(drop=True)
extracted_near = extracted_near.set_index(df0.loc[f_near_out, "id_17"].to_numpy())

In [ ]:
df_out

In [ ]:
df_out.loc[f_near_out, my_columns] = extracted_near
df_out

In [ ]:
# Save to csv
df_out.to_csv(f".\\results_monthly_nearest\\ZV2017-2018_s2-rgbn_monthly.csv", index=False)

## 2.2 Year 2019 s2-rgbn (DONE)

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

Load the `nearest_ids` data:

In [ ]:
%%time
# Nearest IDs shapefile
df0 = pd.read_csv(".\\results_ids\\ZV2017_nearest_ids_19_ajda.csv")

print(df0.shape)
df0.head()

Split the nearest_IDs dataset into "FAR" and "NEAR" sub-datasets. Those that are further than 10m (one pixel) away will be calculated as mean of the nearest 3 samples. Those that are near will be assume the value of the nearest field.

In [ ]:
# Create filters for the full data frame
crop_types = [4, 5, 20, 30, 100, 208, 405, 699, 800]

f_far_out = (df0["SIFRA_KMRS"].isin(crop_types)) & (df0["dist0"] >= 10)
f_near_out = (df0["SIFRA_KMRS"].isin(crop_types)) & (df0["dist0"] < 10)

f_far_out

In [ ]:
# Filter the nearest_IDs data frame
df0 = df0[df0["SIFRA_KMRS"].isin(crop_types)]

# Create filters for the already filtered data frame
f_far_df0 = df0["dist0"] >= 10
f_near_df0 = df0["dist0"] < 10

df0.head()

Load 2019 data:

In [ ]:
%%time
# Monthly S2-rgbn for ear 2019
df1 = pd.read_csv(".\\results_monthly\\KMRS2019_s2-rgbn_monthly.csv")
df1 = df1.set_index("id_19")

print(df1.shape)
df1.head(2)

Prepare output table (same format as 2017 monthly):

In [ ]:
%%time

# Open monthly S2-indices for year 2017 and set all values to nan
df_out = pd.read_csv(".\\results_monthly\\ZV2017_s2-rgbn_monthly.csv")

# Set all values to NaN
df_out.iloc[:, 1:] = np.nan

print(df_out.shape)
df_out.head(2)

### Calculate "far" values

In [ ]:
# Find all FAR and "melt" them
melted = df0.loc[f_far_out, ["id_17", "dist0", "near0", "near1", "near2"]].melt(id_vars=["id_17", "dist0"])
melted

In [ ]:
# List of all column labels from 2018 data frame (without id_18)
my_columns = df1.iloc[:, 0:].columns.to_list()
my_columns

In [ ]:
# Create new data frame based on the index from melted
extracted_values = df1.loc[melted.value.to_numpy(), my_columns].reset_index(drop=True)
extracted_values

In [ ]:
# Join the new data frame with the melted data frame
before_pivot = pd.concat([melted, extracted_values], axis=1)
before_pivot = before_pivot.drop(columns=["value", "dist0"])
before_pivot

In [ ]:
before_pivot.pivot_table(
    values=my_columns,
    index=['id_17'],
    aggfunc=np.nanmean
)

In [ ]:
# Pivot the table back to the original shape by calculating mean of the three nearest data points and insert into output df
df_out.loc[f_far_out, my_columns] = before_pivot.pivot_table(
    values=my_columns,
    index=['id_17'],
    aggfunc=np.nanmean
)

df_out.loc[f_far_out, my_columns]

### Calculate "near" values

In [ ]:
# Find all NEAR and "clean" the data frame
near = df0.loc[f_near_out, ["id_17", "dist0", "near0"]]
near

In [ ]:
extracted_near = df1.loc[near.near0.to_numpy(), my_columns].reset_index(drop=True)
extracted_near = extracted_near.set_index(df0.loc[f_near_out, "id_17"].to_numpy())

In [ ]:
df_out

In [ ]:
df_out.loc[f_near_out, my_columns] = extracted_near
df_out

In [ ]:
# Save to csv
df_out.to_csv(f".\\results_monthly_nearest\\ZV2017-2019_s2-rgbn_monthly.csv", index=False)

# 3. Monthly values of S1-slc

## 2.1 Year 2018 s1_slc

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

Load the `nearest_ids` data:

In [2]:
%%time
# Nearest IDs shapefile
df0 = pd.read_csv(".\\results_ids\\ZV2017_nearest_ids_18_ajda.csv")

print(df0.shape)
df0.head()

(664138, 8)
Wall time: 530 ms


,id_17,GERK_PID,POLJINA_ID,SIFRA_KMRS,dist0,near0,near1,near2
0,0,2537988,2104906,801,72.154793,133377.0,133640.0,468680.0
1,1,2537988,2104908,5,31.536706,133193.0,455175.0,439259.0
2,2,2537988,2104910,203,608.886482,292837.0,228683.0,132731.0
3,3,1143420,2105003,801,103.529690,133367.0,460957.0,133374.0
4,4,3077991,2652377,100,0.000016,126401.0,2978.0,223405.0


Split the nearest_IDs dataset into "FAR" and "NEAR" sub-datasets. Those that are further than 10m (one pixel) away will be calculated as mean of the nearest 3 samples. Those that are near will be assume the value of the nearest field.

In [3]:
# Create filters for the full data frame
crop_types = [4, 5, 20, 30, 100, 208, 405, 699, 800]

f_far_out = (df0["SIFRA_KMRS"].isin(crop_types)) & (df0["dist0"] >= 10)
f_near_out = (df0["SIFRA_KMRS"].isin(crop_types)) & (df0["dist0"] < 10)

f_far_out

0         False
1          True
2         False
3         False
4         False
          ...  
664133    False
664134    False
664135    False
664136    False
664137    False
Length: 664138, dtype: bool

In [4]:
# Filter the nearest_IDs data frame
df0 = df0[df0["SIFRA_KMRS"].isin(crop_types)]

# Create filters for the already filtered data frame
f_far_df0 = df0["dist0"] >= 10
f_near_df0 = df0["dist0"] < 10

df0.head()

,id_17,GERK_PID,POLJINA_ID,SIFRA_KMRS,dist0,near0,near1,near2
1,1,2537988,2104908,5,3.153671e+01,133193.0,455175.0,439259.0
4,4,3077991,2652377,100,1.590166e-05,126401.0,2978.0,223405.0
5,5,3078166,2652382,100,1.729459e-08,126403.0,95191.0,127667.0
6,6,283328,2652383,100,2.876625e-08,126404.0,162454.0,127159.0
7,7,283323,2652384,100,2.805786e-08,126405.0,151293.0,127159.0


Load 2018 data:

In [6]:
%%time
# Monthly S2-indices for ear 2018
df1 = pd.read_csv(".\\results_monthly\\ZV2018_s1-slc_monthly.csv")
df1 = df1.set_index("id_18")

print(df1.shape)
df1.head(2)

(114628, 56)
Wall time: 1.46 s


,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,SIG_ASC_VV_05,SIG_ASC_VV_06,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
id_18,,,,,,,,,,,,,,,,,,,,,
1,0.017827,0.023738,0.026167,0.033239,0.034944,0.018991,0.019597,0.128212,0.221002,0.174808,...,0.274968,0.348063,0.315984,0.448556,0.339626,0.335113,0.342336,0.292151,0.418720,0.433162
6,0.015448,0.019575,0.023196,0.026716,0.026017,0.028838,0.026320,0.106037,0.076187,0.105223,...,0.255912,0.255903,0.321825,0.576367,0.345030,0.310340,0.322201,0.312165,0.266165,0.438230


Prepare output table (same format as 2017 monthly):

In [7]:
%%time

# Open monthly S2-indices for year 2017 and set all values to nan
df_out = pd.read_csv(".\\results_monthly\\ZV2017_s1-slc_monthly.csv")

# Set all values to NaN
df_out.iloc[:, 1:] = np.nan

print(df_out.shape)
df_out.head(2)

(664138, 57)
Wall time: 7.82 s


,id_17,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,SIG_ASC_VV_05,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Calculate "far" values

In [8]:
# Find all FAR and "melt" them
melted = df0.loc[f_far_out, ["id_17", "dist0", "near0", "near1", "near2"]].melt(id_vars=["id_17", "dist0"])
melted

,id_17,dist0,variable,value
0,1,31.536706,near0,133193.0
1,11,256.984082,near0,211359.0
2,51,88.456520,near0,133368.0
3,52,17.583295,near0,133368.0
4,54,212.937327,near0,295963.0
...,...,...,...,...
174826,664067,202.839403,near2,198531.0
174827,664072,210.396471,near2,609606.0
174828,664095,76.520979,near2,110286.0
174829,664112,64.439271,near2,566708.0


In [9]:
# List of all column labels from 2018 data frame (without id_18)
my_columns = df1.iloc[:, 0:].columns.to_list()
my_columns

['SIG_ASC_VH_04',
 'SIG_ASC_VH_05',
 'SIG_ASC_VH_06',
 'SIG_ASC_VH_07',
 'SIG_ASC_VH_08',
 'SIG_ASC_VH_09',
 'SIG_ASC_VH_10',
 'SIG_ASC_VV_04',
 'SIG_ASC_VV_05',
 'SIG_ASC_VV_06',
 'SIG_ASC_VV_07',
 'SIG_ASC_VV_08',
 'SIG_ASC_VV_09',
 'SIG_ASC_VV_10',
 'SIG_DES_VH_04',
 'SIG_DES_VH_05',
 'SIG_DES_VH_06',
 'SIG_DES_VH_07',
 'SIG_DES_VH_08',
 'SIG_DES_VH_09',
 'SIG_DES_VH_10',
 'SIG_DES_VV_04',
 'SIG_DES_VV_05',
 'SIG_DES_VV_06',
 'SIG_DES_VV_07',
 'SIG_DES_VV_08',
 'SIG_DES_VV_09',
 'SIG_DES_VV_10',
 'COH_ASC_VH_04',
 'COH_ASC_VH_05',
 'COH_ASC_VH_06',
 'COH_ASC_VH_07',
 'COH_ASC_VH_08',
 'COH_ASC_VH_09',
 'COH_ASC_VH_10',
 'COH_ASC_VV_04',
 'COH_ASC_VV_05',
 'COH_ASC_VV_06',
 'COH_ASC_VV_07',
 'COH_ASC_VV_08',
 'COH_ASC_VV_09',
 'COH_ASC_VV_10',
 'COH_DES_VH_04',
 'COH_DES_VH_05',
 'COH_DES_VH_06',
 'COH_DES_VH_07',
 'COH_DES_VH_08',
 'COH_DES_VH_09',
 'COH_DES_VH_10',
 'COH_DES_VV_04',
 'COH_DES_VV_05',
 'COH_DES_VV_06',
 'COH_DES_VV_07',
 'COH_DES_VV_08',
 'COH_DES_VV_09',
 'COH_DES_

In [10]:
# Create new data frame based on the index from melted
extracted_values = df1.loc[melted.value.to_numpy(), my_columns].reset_index(drop=True)
extracted_values

,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,SIG_ASC_VV_05,SIG_ASC_VV_06,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
0,0.014038,0.016652,0.026068,0.033162,0.026888,0.023534,0.017635,0.103180,0.092912,0.088684,...,0.276944,0.303573,0.435284,0.610300,0.480342,0.279811,0.299027,0.284886,0.370461,0.529648
1,0.022548,0.019318,0.022866,0.026371,0.026601,0.026096,0.024863,0.084727,0.093011,0.106707,...,0.274019,0.306360,0.297942,0.338337,0.396265,0.284606,0.262420,0.289651,0.276605,0.284236
2,0.015768,0.015507,0.023562,0.028171,0.025286,0.020970,0.017770,0.091484,0.074063,0.095419,...,0.307981,0.308949,0.378243,0.586098,0.477739,0.278404,0.306456,0.292040,0.336564,0.491163
3,0.015768,0.015507,0.023562,0.028171,0.025286,0.020970,0.017770,0.091484,0.074063,0.095419,...,0.307981,0.308949,0.378243,0.586098,0.477739,0.278404,0.306456,0.292040,0.336564,0.491163
4,0.012875,0.013342,0.028967,0.025689,0.025601,0.032757,0.021920,0.088486,0.092539,0.118390,...,0.279501,0.302048,0.350040,0.377459,0.372560,0.261098,0.309908,0.338640,0.367430,0.404612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174826,0.013819,0.011620,0.026105,0.028802,0.026437,0.030270,0.020336,0.101547,0.065334,0.079003,...,0.296951,0.325423,0.413321,0.561558,0.572059,0.288086,0.271803,0.284272,0.415486,0.463756
174827,0.011632,0.024234,0.030451,0.024931,0.026399,0.022691,0.019352,0.086393,0.080366,0.084101,...,0.311032,0.291738,0.333932,0.326910,0.323754,0.310779,0.304895,0.266278,0.274701,0.346214
174828,0.011434,0.024602,0.025631,0.027888,0.020181,0.019743,0.019598,0.091665,0.151274,0.123285,...,0.250764,0.304453,0.284392,0.436377,0.304704,0.294606,0.271078,0.279058,0.285779,0.306187
174829,0.020733,0.016845,0.022454,0.021026,0.020588,0.019529,0.019139,0.105757,0.077695,0.106084,...,0.290989,0.365583,0.366714,0.391170,0.411048,0.291125,0.286214,0.332764,0.371189,0.413958


In [11]:
# Join the new data frame with the melted data frame
before_pivot = pd.concat([melted, extracted_values], axis=1)
before_pivot = before_pivot.drop(columns=["value", "dist0"])
before_pivot

,id_17,variable,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
0,1,near0,0.014038,0.016652,0.026068,0.033162,0.026888,0.023534,0.017635,0.103180,...,0.276944,0.303573,0.435284,0.610300,0.480342,0.279811,0.299027,0.284886,0.370461,0.529648
1,11,near0,0.022548,0.019318,0.022866,0.026371,0.026601,0.026096,0.024863,0.084727,...,0.274019,0.306360,0.297942,0.338337,0.396265,0.284606,0.262420,0.289651,0.276605,0.284236
2,51,near0,0.015768,0.015507,0.023562,0.028171,0.025286,0.020970,0.017770,0.091484,...,0.307981,0.308949,0.378243,0.586098,0.477739,0.278404,0.306456,0.292040,0.336564,0.491163
3,52,near0,0.015768,0.015507,0.023562,0.028171,0.025286,0.020970,0.017770,0.091484,...,0.307981,0.308949,0.378243,0.586098,0.477739,0.278404,0.306456,0.292040,0.336564,0.491163
4,54,near0,0.012875,0.013342,0.028967,0.025689,0.025601,0.032757,0.021920,0.088486,...,0.279501,0.302048,0.350040,0.377459,0.372560,0.261098,0.309908,0.338640,0.367430,0.404612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174826,664067,near2,0.013819,0.011620,0.026105,0.028802,0.026437,0.030270,0.020336,0.101547,...,0.296951,0.325423,0.413321,0.561558,0.572059,0.288086,0.271803,0.284272,0.415486,0.463756
174827,664072,near2,0.011632,0.024234,0.030451,0.024931,0.026399,0.022691,0.019352,0.086393,...,0.311032,0.291738,0.333932,0.326910,0.323754,0.310779,0.304895,0.266278,0.274701,0.346214
174828,664095,near2,0.011434,0.024602,0.025631,0.027888,0.020181,0.019743,0.019598,0.091665,...,0.250764,0.304453,0.284392,0.436377,0.304704,0.294606,0.271078,0.279058,0.285779,0.306187
174829,664112,near2,0.020733,0.016845,0.022454,0.021026,0.020588,0.019529,0.019139,0.105757,...,0.290989,0.365583,0.366714,0.391170,0.411048,0.291125,0.286214,0.332764,0.371189,0.413958


In [12]:
before_pivot.pivot_table(
    values=my_columns,
    index=['id_17'],
    aggfunc=np.nanmean
)

,COH_ASC_VH_04,COH_ASC_VH_05,COH_ASC_VH_06,COH_ASC_VH_07,COH_ASC_VH_08,COH_ASC_VH_09,COH_ASC_VH_10,COH_ASC_VV_04,COH_ASC_VV_05,COH_ASC_VV_06,...,SIG_DES_VH_08,SIG_DES_VH_09,SIG_DES_VH_10,SIG_DES_VV_04,SIG_DES_VV_05,SIG_DES_VV_06,SIG_DES_VV_07,SIG_DES_VV_08,SIG_DES_VV_09,SIG_DES_VV_10
id_17,,,,,,,,,,,,,,,,,,,,,
1,0.376895,0.288692,0.279950,0.305743,0.301909,0.320272,0.378572,0.514180,0.402201,0.291540,...,0.031894,0.025514,0.024388,0.167186,0.089452,0.104028,0.141676,0.137460,0.112230,0.129914
11,0.297279,0.303046,0.279870,0.282678,0.318263,0.297713,0.283745,0.401727,0.407684,0.336432,...,0.022422,0.023956,0.026768,0.112448,0.110417,0.126318,0.118699,0.093242,0.093714,0.101929
51,0.355543,0.293067,0.285473,0.291773,0.317533,0.339860,0.409825,0.472942,0.394140,0.281168,...,0.032955,0.029196,0.024551,0.134676,0.097341,0.111622,0.129954,0.144096,0.114726,0.108830
52,0.355543,0.293067,0.285473,0.291773,0.317533,0.339860,0.409825,0.472942,0.394140,0.281168,...,0.032955,0.029196,0.024551,0.134676,0.097341,0.111622,0.129954,0.144096,0.114726,0.108830
54,0.309587,0.267885,0.284444,0.324336,0.342417,0.404480,0.408843,0.402204,0.351237,0.273201,...,0.025634,0.028287,0.022274,0.099035,0.078902,0.119540,0.119905,0.114427,0.121841,0.118495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664067,0.359337,0.297043,0.261649,0.263689,0.277736,0.407425,0.412455,0.526483,0.449528,0.276230,...,0.034772,0.034945,0.020271,0.159487,0.101200,0.120208,0.129217,0.149714,0.145944,0.102404
664072,0.329100,0.344128,0.293747,0.289583,0.316117,0.309711,0.357489,0.402577,0.374272,0.338740,...,0.028475,0.027798,0.026604,0.081785,0.095915,0.100652,0.103667,0.111522,0.114657,0.095184
664095,0.310293,0.269619,0.292206,0.274787,0.277102,0.291227,0.318184,0.455138,0.333505,0.324268,...,0.024551,0.025512,0.026141,0.129606,0.127578,0.124009,0.128452,0.118869,0.132313,0.151049


In [13]:
# Pivot the table back to the original shape by calculating mean of the three nearest data points and insert into output df
df_out.loc[f_far_out, my_columns] = before_pivot.pivot_table(
    values=my_columns,
    index=['id_17'],
    aggfunc=np.nanmean
)

df_out.loc[f_far_out, my_columns]

,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,SIG_ASC_VV_05,SIG_ASC_VV_06,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
1,0.018377,0.016486,0.027031,0.030543,0.026689,0.022702,0.020971,0.111041,0.087099,0.100226,...,0.274526,0.303650,0.385616,0.547654,0.412025,0.283062,0.290619,0.292545,0.351592,0.461500
11,0.020486,0.017850,0.025544,0.026263,0.018284,0.023446,0.022109,0.087498,0.137167,0.225656,...,0.290546,0.288867,0.300019,0.378259,0.381902,0.282748,0.269224,0.329759,0.289861,0.311985
51,0.014815,0.016409,0.023284,0.026045,0.027048,0.030254,0.021183,0.085777,0.085691,0.093854,...,0.282097,0.302524,0.345498,0.499674,0.472846,0.290288,0.288460,0.287904,0.332537,0.438448
52,0.014815,0.016409,0.023284,0.026045,0.027048,0.030254,0.021183,0.085777,0.085691,0.093854,...,0.282097,0.302524,0.345498,0.499674,0.472846,0.290288,0.288460,0.287904,0.332537,0.438448
54,0.013073,0.012272,0.026403,0.026155,0.023785,0.030047,0.015117,0.091219,0.074740,0.106659,...,0.276509,0.289740,0.384738,0.395863,0.361345,0.277346,0.313507,0.311539,0.367065,0.504144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664067,0.015334,0.014377,0.028564,0.029477,0.027348,0.030180,0.020461,0.099562,0.084666,0.091099,...,0.289471,0.340123,0.411728,0.516044,0.528996,0.281920,0.276196,0.287697,0.405096,0.476875
664072,0.017928,0.022199,0.028847,0.027299,0.027938,0.022420,0.020920,0.096440,0.097940,0.109544,...,0.297577,0.281930,0.329907,0.356798,0.354790,0.312926,0.306493,0.301111,0.305029,0.345841
664095,0.013992,0.025369,0.023825,0.025910,0.024375,0.019591,0.020603,0.106392,0.135418,0.115000,...,0.286639,0.330213,0.333451,0.493162,0.339867,0.317485,0.297127,0.300893,0.370770,0.399010
664112,0.020117,0.018023,0.024429,0.024052,0.022548,0.023765,0.020628,0.089485,0.071468,0.094020,...,0.292096,0.318734,0.337697,0.414005,0.373277,0.290049,0.278236,0.307469,0.328143,0.400401


### Calculate "near" values

In [14]:
# Find all NEAR and "clean" the data frame
near = df0.loc[f_near_out, ["id_17", "dist0", "near0"]]
near

,id_17,dist0,near0
4,4,1.590166e-05,126401.0
5,5,1.729459e-08,126403.0
6,6,2.876625e-08,126404.0
7,7,2.805786e-08,126405.0
8,8,2.587472e+00,126407.0
...,...,...,...
664049,664049,4.617919e-08,195799.0
664077,664077,6.404496e-10,572089.0
664088,664088,4.199936e-10,279923.0
664089,664089,9.134553e+00,279925.0


In [15]:
extracted_near = df1.loc[near.near0.to_numpy(), my_columns].reset_index(drop=True)
extracted_near = extracted_near.set_index(df0.loc[f_near_out, "id_17"].to_numpy())

In [16]:
df_out

,id_17,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,SIG_ASC_VV_05,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.018377,0.016486,0.027031,0.030543,0.026689,0.022702,0.020971,0.111041,0.087099,...,0.274526,0.30365,0.385616,0.547654,0.412025,0.283062,0.290619,0.292545,0.351592,0.4615
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664133,664133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664134,664134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664135,664135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664136,664136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_out.loc[f_near_out, my_columns] = extracted_near
df_out

,id_17,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,SIG_ASC_VV_05,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.018377,0.016486,0.027031,0.030543,0.026689,0.022702,0.020971,0.111041,0.087099,...,0.274526,0.303650,0.385616,0.547654,0.412025,0.283062,0.290619,0.292545,0.351592,0.461500
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.019077,0.022900,0.023169,0.025746,0.020636,0.022284,0.024776,0.082769,0.085697,...,0.294413,0.300423,0.326417,0.354737,0.289858,0.302365,0.301940,0.330369,0.296798,0.356478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664133,664133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664134,664134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664135,664135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664136,664136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Save to csv
df_out.to_csv(f".\\results_monthly_nearest\\ZV2017-2018_s1-slc_monthly.csv", index=False)

## 2.2 Year 2019 s1_slc

In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np

Load the `nearest_ids` data:

In [4]:
%%time
# Nearest IDs shapefile
df0 = pd.read_csv(".\\results_ids\\ZV2017_nearest_ids_19_ajda.csv")

print(df0.shape)
df0.head()

(664138, 8)
Wall time: 663 ms


,id_17,GERK_PID,POLJINA_ID,SIFRA_KMRS,dist0,near0,near1,near2
0,0,2537988,2104906,801,44.375468,8378.0,282621.0,191282.0
1,1,2537988,2104908,5,23.064102,7685.0,427672.0,7732.0
2,2,2537988,2104910,203,44.696477,4577.0,485602.0,47790.0
3,3,1143420,2105003,801,35.919025,7693.0,111779.0,47326.0
4,4,3077991,2652377,100,0.358454,573041.0,247312.0,269034.0


Split the nearest_IDs dataset into "FAR" and "NEAR" sub-datasets. Those that are further than 10m (one pixel) away will be calculated as mean of the nearest 3 samples. Those that are near will be assume the value of the nearest field.

In [5]:
# Create filters for the full data frame
crop_types = [4, 5, 20, 30, 100, 208, 405, 699, 800]

f_far_out = (df0["SIFRA_KMRS"].isin(crop_types)) & (df0["dist0"] >= 10)
f_near_out = (df0["SIFRA_KMRS"].isin(crop_types)) & (df0["dist0"] < 10)

f_far_out

0         False
1          True
2         False
3         False
4         False
          ...  
664133    False
664134    False
664135    False
664136    False
664137    False
Length: 664138, dtype: bool

In [6]:
# Filter the nearest_IDs data frame
df0 = df0[df0["SIFRA_KMRS"].isin(crop_types)]

# Create filters for the already filtered data frame
f_far_df0 = df0["dist0"] >= 10
f_near_df0 = df0["dist0"] < 10

df0.head()

,id_17,GERK_PID,POLJINA_ID,SIFRA_KMRS,dist0,near0,near1,near2
1,1,2537988,2104908,5,23.064102,7685.0,427672.0,7732.0
4,4,3077991,2652377,100,0.358454,573041.0,247312.0,269034.0
5,5,3078166,2652382,100,0.354016,573045.0,556171.0,74757.0
6,6,283328,2652383,100,0.380426,573046.0,499275.0,111984.0
7,7,283323,2652384,100,0.380303,573423.0,440736.0,111984.0


Load 2018 data:

In [8]:
%%time
# Monthly S1-slc for year 2019
df1 = pd.read_csv(".\\results_monthly\\KMRS2019_s1-slc_monthly.csv")
df1 = df1.set_index("id_19")

print(df1.shape)
df1.head(2)

(111888, 56)
Wall time: 1.72 s


,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,SIG_ASC_VV_05,SIG_ASC_VV_06,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
id_19,,,,,,,,,,,,,,,,,,,,,
9,0.023094,0.029670,0.023403,0.027684,0.027980,0.02250,0.023654,0.096612,0.135179,0.105678,...,0.284233,0.271960,0.300977,0.290621,0.309541,0.305231,0.289851,0.288277,0.305021,0.322764
12,0.020476,0.028213,0.026821,0.024316,0.024195,0.02311,0.027378,0.098572,0.114729,0.096931,...,0.309345,0.293111,0.346630,0.374705,0.299361,0.294275,0.332639,0.341203,0.367490,0.366752


Prepare output table (same format as 2017 monthly):

In [9]:
%%time

# Open monthly S1-slc for year 2017 and set all values to nan
df_out = pd.read_csv(".\\results_monthly\\ZV2017_s1-slc_monthly.csv")

# Set all values to NaN
df_out.iloc[:, 1:] = np.nan

print(df_out.shape)
df_out.head(2)

(664138, 57)
Wall time: 9.61 s


,id_17,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,SIG_ASC_VV_05,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Calculate "far" values

In [10]:
# Find all FAR and "melt" them
melted = df0.loc[f_far_out, ["id_17", "dist0", "near0", "near1", "near2"]].melt(id_vars=["id_17", "dist0"])
melted

,id_17,dist0,variable,value
0,1,23.064102,near0,7685.0
1,11,36.380763,near0,568136.0
2,52,35.097677,near0,7733.0
3,58,13.827947,near0,7728.0
4,60,19.992856,near0,7732.0
...,...,...,...,...
137125,664058,582.982790,near2,503062.0
137126,664066,667.339087,near2,277459.0
137127,664072,302.544696,near2,307687.0
137128,664095,289.417185,near2,335418.0


In [11]:
# List of all column labels from 2018 data frame (without id_18)
my_columns = df1.iloc[:, 0:].columns.to_list()
my_columns

['SIG_ASC_VH_04',
 'SIG_ASC_VH_05',
 'SIG_ASC_VH_06',
 'SIG_ASC_VH_07',
 'SIG_ASC_VH_08',
 'SIG_ASC_VH_09',
 'SIG_ASC_VH_10',
 'SIG_ASC_VV_04',
 'SIG_ASC_VV_05',
 'SIG_ASC_VV_06',
 'SIG_ASC_VV_07',
 'SIG_ASC_VV_08',
 'SIG_ASC_VV_09',
 'SIG_ASC_VV_10',
 'SIG_DES_VH_04',
 'SIG_DES_VH_05',
 'SIG_DES_VH_06',
 'SIG_DES_VH_07',
 'SIG_DES_VH_08',
 'SIG_DES_VH_09',
 'SIG_DES_VH_10',
 'SIG_DES_VV_04',
 'SIG_DES_VV_05',
 'SIG_DES_VV_06',
 'SIG_DES_VV_07',
 'SIG_DES_VV_08',
 'SIG_DES_VV_09',
 'SIG_DES_VV_10',
 'COH_ASC_VH_04',
 'COH_ASC_VH_05',
 'COH_ASC_VH_06',
 'COH_ASC_VH_07',
 'COH_ASC_VH_08',
 'COH_ASC_VH_09',
 'COH_ASC_VH_10',
 'COH_ASC_VV_04',
 'COH_ASC_VV_05',
 'COH_ASC_VV_06',
 'COH_ASC_VV_07',
 'COH_ASC_VV_08',
 'COH_ASC_VV_09',
 'COH_ASC_VV_10',
 'COH_DES_VH_04',
 'COH_DES_VH_05',
 'COH_DES_VH_06',
 'COH_DES_VH_07',
 'COH_DES_VH_08',
 'COH_DES_VH_09',
 'COH_DES_VH_10',
 'COH_DES_VV_04',
 'COH_DES_VV_05',
 'COH_DES_VV_06',
 'COH_DES_VV_07',
 'COH_DES_VV_08',
 'COH_DES_VV_09',
 'COH_DES_

In [12]:
# Create new data frame based on the index from melted
extracted_values = df1.loc[melted.value.to_numpy(), my_columns].reset_index(drop=True)
extracted_values

,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,SIG_ASC_VV_05,SIG_ASC_VV_06,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
0,0.014610,0.015831,0.019677,0.023528,0.026856,0.037548,0.029112,0.055691,0.085074,0.070087,...,0.287256,0.301774,0.373834,0.394142,0.539228,0.286790,0.291146,0.279210,0.274184,0.408185
1,0.012448,0.022890,0.022195,0.014796,0.015984,0.022042,0.020303,0.058000,0.090919,0.081395,...,0.269587,0.247700,0.275813,0.467019,0.383157,0.243617,0.299788,0.379003,0.296729,0.286744
2,0.019270,0.018131,0.019152,0.024012,0.028511,0.028250,0.018517,0.066735,0.081101,0.073529,...,0.298802,0.305850,0.346804,0.405729,0.444793,0.299212,0.297523,0.304821,0.317798,0.433574
3,0.021925,0.017199,0.025787,0.029476,0.032357,0.030168,0.017901,0.150981,0.080608,0.100197,...,0.270833,0.304468,0.403962,0.451858,0.420949,0.314927,0.279500,0.303291,0.314521,0.498563
4,0.009008,0.013775,0.020410,0.025470,0.029057,0.033278,0.033885,0.064456,0.105112,0.075671,...,0.280528,0.284646,0.359496,0.522255,0.671323,0.330709,0.266182,0.254212,0.277895,0.349990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137125,0.016037,0.022078,0.016305,0.028760,0.028270,0.030496,0.018585,0.103866,0.121035,0.066528,...,0.259914,0.284129,0.359795,0.515151,0.383914,0.330207,0.297535,0.290649,0.305670,0.412499
137126,0.016470,0.022666,0.027663,0.028402,0.025578,0.015954,0.014199,0.114337,0.110071,0.093748,...,0.321152,0.250971,0.310654,0.314540,0.367164,0.261198,0.288925,0.263698,0.398712,0.446741
137127,0.026758,0.027578,0.023129,0.044133,0.036012,0.042749,0.036513,0.153084,0.124711,0.111850,...,0.281786,0.271530,0.295424,0.597541,0.471094,0.413426,0.357378,0.373262,0.371743,0.443621
137128,0.015521,0.025252,0.019575,0.016086,0.020475,0.019619,0.017310,0.179453,0.161972,0.083644,...,0.246658,0.273502,0.311586,0.432518,0.308769,0.260044,0.344135,0.285597,0.267173,0.289415


In [13]:
# Join the new data frame with the melted data frame
before_pivot = pd.concat([melted, extracted_values], axis=1)
before_pivot = before_pivot.drop(columns=["value", "dist0"])
before_pivot

,id_17,variable,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
0,1,near0,0.014610,0.015831,0.019677,0.023528,0.026856,0.037548,0.029112,0.055691,...,0.287256,0.301774,0.373834,0.394142,0.539228,0.286790,0.291146,0.279210,0.274184,0.408185
1,11,near0,0.012448,0.022890,0.022195,0.014796,0.015984,0.022042,0.020303,0.058000,...,0.269587,0.247700,0.275813,0.467019,0.383157,0.243617,0.299788,0.379003,0.296729,0.286744
2,52,near0,0.019270,0.018131,0.019152,0.024012,0.028511,0.028250,0.018517,0.066735,...,0.298802,0.305850,0.346804,0.405729,0.444793,0.299212,0.297523,0.304821,0.317798,0.433574
3,58,near0,0.021925,0.017199,0.025787,0.029476,0.032357,0.030168,0.017901,0.150981,...,0.270833,0.304468,0.403962,0.451858,0.420949,0.314927,0.279500,0.303291,0.314521,0.498563
4,60,near0,0.009008,0.013775,0.020410,0.025470,0.029057,0.033278,0.033885,0.064456,...,0.280528,0.284646,0.359496,0.522255,0.671323,0.330709,0.266182,0.254212,0.277895,0.349990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137125,664058,near2,0.016037,0.022078,0.016305,0.028760,0.028270,0.030496,0.018585,0.103866,...,0.259914,0.284129,0.359795,0.515151,0.383914,0.330207,0.297535,0.290649,0.305670,0.412499
137126,664066,near2,0.016470,0.022666,0.027663,0.028402,0.025578,0.015954,0.014199,0.114337,...,0.321152,0.250971,0.310654,0.314540,0.367164,0.261198,0.288925,0.263698,0.398712,0.446741
137127,664072,near2,0.026758,0.027578,0.023129,0.044133,0.036012,0.042749,0.036513,0.153084,...,0.281786,0.271530,0.295424,0.597541,0.471094,0.413426,0.357378,0.373262,0.371743,0.443621
137128,664095,near2,0.015521,0.025252,0.019575,0.016086,0.020475,0.019619,0.017310,0.179453,...,0.246658,0.273502,0.311586,0.432518,0.308769,0.260044,0.344135,0.285597,0.267173,0.289415


In [14]:
before_pivot.pivot_table(
    values=my_columns,
    index=['id_17'],
    aggfunc=np.nanmean
)

,COH_ASC_VH_04,COH_ASC_VH_05,COH_ASC_VH_06,COH_ASC_VH_07,COH_ASC_VH_08,COH_ASC_VH_09,COH_ASC_VH_10,COH_ASC_VV_04,COH_ASC_VV_05,COH_ASC_VV_06,...,SIG_DES_VH_08,SIG_DES_VH_09,SIG_DES_VH_10,SIG_DES_VV_04,SIG_DES_VV_05,SIG_DES_VV_06,SIG_DES_VV_07,SIG_DES_VV_08,SIG_DES_VV_09,SIG_DES_VV_10
id_17,,,,,,,,,,,,,,,,,,,,,
1,0.313674,0.411328,0.282740,0.270753,0.260857,0.358477,0.384438,0.396395,0.582249,0.322560,...,0.032754,0.032087,0.029577,0.073072,0.101445,0.098366,0.130313,0.139269,0.124218,0.121427
11,0.282858,0.294848,0.260968,0.309894,0.287505,0.286152,0.284643,0.358339,0.329405,0.292854,...,0.020504,0.024723,0.027044,0.102971,0.125555,0.103017,0.106454,0.092807,0.113591,0.126185
52,0.354120,0.341165,0.293808,0.283325,0.290957,0.339959,0.367153,0.420506,0.478312,0.307545,...,0.028391,0.025379,0.023959,0.104101,0.094622,0.102709,0.108575,0.117987,0.106096,0.108162
58,0.393255,0.336139,0.312542,0.275564,0.270678,0.348937,0.362217,0.455839,0.418386,0.314343,...,0.030143,0.028311,0.025281,0.121916,0.092938,0.115060,0.120459,0.128192,0.112602,0.111891
60,0.313674,0.411328,0.282740,0.270753,0.260857,0.358477,0.384438,0.396395,0.582249,0.322560,...,0.032754,0.032087,0.029577,0.073072,0.101445,0.098366,0.130313,0.139269,0.124218,0.121427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664058,0.360912,0.356790,0.319797,0.284786,0.275853,0.347531,0.369590,0.481692,0.475149,0.352276,...,0.025410,0.026519,0.023562,0.084171,0.091134,0.070984,0.092248,0.093164,0.111364,0.125677
664066,0.436378,0.461800,0.379719,0.350590,0.345449,0.407034,0.443914,0.484548,0.545539,0.381650,...,0.025567,0.017279,0.015613,0.071022,0.074329,0.079699,0.099997,0.090820,0.073078,0.074588
664072,0.273132,0.300813,0.285258,0.283848,0.302135,0.344564,0.310129,0.324418,0.387262,0.321242,...,0.025033,0.026602,0.025361,0.089483,0.086353,0.104378,0.117883,0.106244,0.100065,0.107654


In [15]:
# Pivot the table back to the original shape by calculating mean of the three nearest data points and insert into output df
df_out.loc[f_far_out, my_columns] = before_pivot.pivot_table(
    values=my_columns,
    index=['id_17'],
    aggfunc=np.nanmean
)

df_out.loc[f_far_out, my_columns]

,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,SIG_ASC_VV_05,SIG_ASC_VV_06,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
1,0.013331,0.017100,0.019724,0.024522,0.027806,0.035084,0.032611,0.063672,0.109540,0.077903,...,0.277479,0.297105,0.384600,0.407934,0.606099,0.319604,0.277397,0.266686,0.288290,0.409081
11,0.012428,0.020690,0.020495,0.015072,0.017935,0.019718,0.018763,0.059093,0.087262,0.073075,...,0.304207,0.303526,0.286072,0.463795,0.407871,0.331480,0.379095,0.390929,0.353496,0.327771
52,0.019631,0.016197,0.020013,0.024097,0.028324,0.029866,0.022449,0.100045,0.089807,0.078001,...,0.287336,0.302687,0.356693,0.461314,0.505603,0.334169,0.279439,0.291751,0.307915,0.416072
58,0.021946,0.018164,0.024901,0.027485,0.029532,0.030955,0.022478,0.129276,0.087704,0.092860,...,0.287921,0.309316,0.364474,0.469175,0.451086,0.307894,0.295589,0.318390,0.318848,0.432304
60,0.013331,0.017100,0.019724,0.024522,0.027806,0.035084,0.032611,0.063672,0.109540,0.077903,...,0.277479,0.297105,0.384600,0.407934,0.606099,0.319604,0.277397,0.266686,0.288290,0.409081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664058,0.017590,0.020227,0.015574,0.025519,0.025443,0.027769,0.018232,0.090854,0.104273,0.064403,...,0.277090,0.352814,0.404645,0.510281,0.465335,0.359067,0.292686,0.290933,0.365747,0.475295
664066,0.076305,0.145042,0.099993,0.104130,0.107319,0.100877,0.072122,0.237884,0.235446,0.266127,...,0.322747,0.342018,0.368031,0.398452,0.432366,0.313965,0.289258,0.291466,0.406281,0.471928
664072,0.018762,0.023027,0.018142,0.030000,0.026569,0.029986,0.024407,0.084383,0.105059,0.080978,...,0.298078,0.299199,0.323332,0.409441,0.414792,0.333683,0.320326,0.319452,0.340863,0.373857
664095,0.013789,0.022962,0.021474,0.022184,0.022170,0.022820,0.018389,0.123107,0.159017,0.096316,...,0.264781,0.292239,0.318074,0.422582,0.365865,0.290841,0.313254,0.280670,0.292997,0.360445


### Calculate "near" values

In [16]:
# Find all NEAR and "clean" the data frame
near = df0.loc[f_near_out, ["id_17", "dist0", "near0"]]
near

,id_17,dist0,near0
4,4,0.358454,573041.0
5,5,0.354016,573045.0
6,6,0.380426,573046.0
7,7,0.380303,573423.0
8,8,2.397666,573043.0
...,...,...,...
664077,664077,0.239102,570996.0
664088,664088,0.104019,224764.0
664089,664089,9.135624,224765.0
664101,664101,0.349822,615916.0


In [17]:
extracted_near = df1.loc[near.near0.to_numpy(), my_columns].reset_index(drop=True)
extracted_near = extracted_near.set_index(df0.loc[f_near_out, "id_17"].to_numpy())

In [18]:
df_out

,id_17,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,SIG_ASC_VV_05,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.013331,0.0171,0.019724,0.024522,0.027806,0.035084,0.032611,0.063672,0.10954,...,0.277479,0.297105,0.3846,0.407934,0.606099,0.319604,0.277397,0.266686,0.28829,0.409081
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664133,664133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664134,664134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664135,664135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664136,664136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_out.loc[f_near_out, my_columns] = extracted_near
df_out

,id_17,SIG_ASC_VH_04,SIG_ASC_VH_05,SIG_ASC_VH_06,SIG_ASC_VH_07,SIG_ASC_VH_08,SIG_ASC_VH_09,SIG_ASC_VH_10,SIG_ASC_VV_04,SIG_ASC_VV_05,...,COH_DES_VH_08,COH_DES_VH_09,COH_DES_VH_10,COH_DES_VV_04,COH_DES_VV_05,COH_DES_VV_06,COH_DES_VV_07,COH_DES_VV_08,COH_DES_VV_09,COH_DES_VV_10
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.013331,0.017100,0.019724,0.024522,0.027806,0.035084,0.032611,0.063672,0.109540,...,0.277479,0.297105,0.38460,0.407934,0.606099,0.319604,0.277397,0.266686,0.288290,0.409081
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.014656,0.017438,0.026378,0.024885,0.024495,0.024632,0.024367,0.081208,0.087159,...,0.302242,0.322055,0.33753,0.458995,0.419668,0.341678,0.359266,0.364299,0.409735,0.430976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664133,664133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664134,664134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664135,664135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664136,664136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Save to csv
df_out.to_csv(f".\\results_monthly_nearest\\ZV2017-2019_s1-slc_monthly.csv", index=False)